**Project Introduction: SwiftOnSecurity Sysmon Configuration for Splunk Detection**

This project builds upon a structured progression of Windows telemetry enhancement efforts for cybersecurity detection. Initially, we worked with native Windows audit policies, manually tuning them to reduce noise and improve investigative fidelity. However, audit policy alone produced overwhelming and inconsistent results, particularly for behavioral detections involving lateral movement, privilege escalation, and process chaining.

We then introduced Sysmon, a lightweight system monitoring tool from Microsoft Sysinternals, and installed it with a default configuration. This significantly improved our visibility by capturing detailed process creation events, network connections, and command-line activity. The benefit was immediately clear: we gained visibility into attacker behaviors that native audit policy alone could not expose.

Now, we move to the next stage — implementing a vetted, tuned Sysmon configuration developed by SwiftOnSecurity, a well-regarded security practitioner. This configuration is designed to reduce noise, focus on high-fidelity telemetry, and simulate the tuned visibility typically seen in Security Operations Center (SOC) environments. It is optimized specifically for SIEM ingestion, making it ideal for Splunk deployments.

Advantages of Using the SwiftOnSecurity Configuration
🚫 Suppresses Low-Value Events: Reduces volume by filtering redundant or noisy system activity (e.g., excessive svchost.exe, frequent benign processes).

🔍 Preserves Investigative Context: Retains command-line arguments, parent-child process chains, and hash data to support full-scope investigations.

🛠️ Focuses on Known Tactics: Prioritizes telemetry aligned with known attacker techniques (e.g., PowerShell, scheduled tasks, RDP, SMB, suspicious binaries).

⚙️ Mimics SOC-Ready Setup: Mirrors the type of Sysmon tuning that enterprise defenders apply in production SIEM environments.

This configuration significantly shifts the telemetry from raw verbosity toward actionable signal, enabling more efficient and focused detection workflows. Unlike default Sysmon or Windows audit settings, it suppresses repetitive, low-value events while maintaining coverage of high-priority attack behaviors and necessary context (e.g., hashes, parent-child relationships). This approach reflects real-world SIEM tuning strategies used in enterprise SOCs, allowing defenders to focus on what matters without losing visibility into attacker techniques that exploit native tools or benign-looking processes.



**SwiftOnSecurity Configuration and Deployment Summary**

To enhance the fidelity and efficiency of Windows event telemetry within Splunk, the SwiftOnSecurity Sysmon configuration was selected for deployment. This widely adopted configuration offers curated rules that prioritize high-value signal while reducing unnecessary noise, making it well-suited for detection engineering and operational monitoring.

The process began by downloading the latest sysmonconfig-export.xml file from the official SwiftOnSecurity GitHub repository. The configuration was saved locally and applied to the system using the Sysmon command-line utility:
**C:\Windows\Sysmon64.exe -c "C:\Users\Steve\Desktop\sysmonconfig-export.xml"**

Sysmon validated the configuration successfully, confirming that it was parsed correctly and applied. This activated a refined event logging policy designed to capture detailed process activity, command-line arguments, hash values, parent-child relationships, and other relevant indicators of execution behavior.

Following configuration, test commands were executed in PowerShell to validate telemetry coverage. For example, launching Notepad generated a corresponding Event ID 1 entry in the Microsoft-Windows-Sysmon/Operational log. This confirmed the capture of enriched fields such as image path, command-line string, process hashes, and parent process metadata — all consistent with SwiftOnSecurity's enhanced logging schema.

To confirm ingestion into Splunk, the system was restarted, and incoming events were verified. A 15-minute snapshot showed over 1,000 Sysmon events indexed, demonstrating both the active state of telemetry and Splunk’s ability to receive, parse, and display the enhanced data.

These steps establish a functional and efficient telemetry baseline for future detection efforts. The system is now operating with SwiftOnSecurity's configuration, providing greater clarity, reduced noise, and a foundation aligned with enterprise SOC standards.



✅ **Obfuscated PowerShell Execution: Visibility Test and Validation**

To validate the effectiveness of the newly deployed SwiftOnSecurity Sysmon configuration, we conducted a test targeting one of the historically weak areas in native Windows logging: obfuscated PowerShell execution. Specifically, we ran a base64-encoded PowerShell command from a standard (non-elevated) PowerShell terminal. The command used the -EncodedCommand flag to invoke Start-Process notepad.exe, simulating a simple but realistic example of evasive execution.

After executing the obfuscated command, we used Get-WinEvent in PowerShell to query the Microsoft-Windows-Sysmon/Operational log for Event ID 1 (Process Create) where the process image or command line referenced notepad.exe. The results confirmed that Sysmon had captured full command line visibility, including the obfuscated -EncodedCommand string. Additionally, the telemetry showed a complete process chain — linking the parent PowerShell process to the spawned Notepad executable — along with key forensic artifacts such as hashes, user account, logon ID, integrity level, and GUIDs.

This successful capture demonstrates that the SwiftOnSecurity configuration provides high-fidelity visibility into obfuscated PowerShell activity, even when executed without administrative privileges. In prior projects using native Windows audit logging (Event ID 4688), this level of detail was inconsistent or entirely absent — especially for encoded command lines. Sysmon fills that detection gap reliably.

Next, we will confirm that this telemetry has also been successfully ingested and indexed by Splunk, thereby validating end-to-end detection readiness across both endpoint and SIEM layers.



🧪 **Obfuscated PowerShell Execution: Visibility Test and SIEM Validation**

To validate our upgraded telemetry pipeline, we executed an obfuscated PowerShell command on the host designed to launch notepad.exe. This test simulated a basic attacker behavior — code execution via encoded command line — which often evades traditional logging mechanisms when visibility is incomplete.

The obfuscated command was executed using:
powershell.exe -EncodedCommand UwB0AGEAcgB0AC0AUAByAG8AYwBlAHMAcwAgAG4AbwB0AGUAcABhAGQALgBlAHgAZQA=

This successfully triggered a Sysmon Event ID 1 (Process Create) log, capturing the full decoded execution, including both the Image (notepad.exe) and the ParentCommandLine (powershell.exe with the base64-encoded string). The visibility confirmed that:

The SwiftOnSecurity Sysmon configuration actively captured obfuscated execution paths.

Command-line arguments were logged in full.

Parent-child relationships were correctly preserved.

IntegrityLevel and LogonId metadata were available for investigative correlation.

✅ **SIEM Integration**
We verified that this process creation event was successfully ingested into Splunk, where it appeared under the expected source type:


sourcetype = XmlWinEventLog:Microsoft-Windows-Sysmon/Operational
index = main
A filtered search for notepad.exe revealed the correct telemetry, including execution time, user context, and full command line. This confirmed that end-to-end visibility was operational from endpoint telemetry generation (Sysmon) to centralized SIEM ingestion and indexing (Splunk).

✅ **Project Conclusion**
This streamlined project demonstrated how to:

Deploy a hardened Sysmon configuration for high-fidelity detection.

Verify Windows event telemetry using both PowerShell and Splunk.

Detect obfuscated execution paths through process creation events.

Confirm SIEM visibility through live searches within Splunk.

This telemetry upgrade lays the groundwork for more advanced detections, including:

Credential access attempts,

Scheduled task abuse,

Remote execution,

Beaconing patterns, and more.